# Reranking file

## Imports

In [ ]:
from src.route_finder import find_rotues


## Graph simulation of routes

In [ ]:
graph = {
    "15L": ["A","B","C","D","E"],
    "0": ["F","C","G"],
    "1": ["H","I","J","E"],
}